In [10]:
import numpy as np
from sklearn.cluster import KMeans
from collections import defaultdict
from math import radians, cos, sin, asin, sqrt
import sqlite3

# -------------------------------
# 📍 Race Locations (lat, lon)
# Add or modify as needed
# -------------------------------
# Connect to the database
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Query to fetch race locations and their lat/lon
query = """
SELECT fc.circuit, fg.latitude, fg.longitude
FROM fone_calendar fc
JOIN fone_geography fg ON fc.geo_id = fg.id
WHERE fc.year = 2025
"""
cursor.execute(query)

# Build the race_locations dictionary
race_locations = {row[0]: (row[1], row[2]) for row in cursor.fetchall()}

# Close the database connection
conn.close()

track_names = list(race_locations.keys())
locations = np.array(list(race_locations.values()))

# -------------------------------
# 🌍 Haversine Distance Function
# -------------------------------
def haversine(loc1, loc2):
    lat1, lon1 = loc1
    lat2, lon2 = loc2
    # convert to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return 6371 * c  # Radius of Earth in km

# -------------------------------
# Step 1: Cluster the Races
# -------------------------------
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(locations)
labels = kmeans.labels_

clusters = defaultdict(list)
for i, label in enumerate(labels):
    clusters[label].append(track_names[i])

# -------------------------------
# Step 2: TSP within Clusters
# -------------------------------
def nearest_neighbor_tsp(track_list):
    unvisited = track_list[:]
    path = [unvisited.pop(0)]
    while unvisited:
        last = path[-1]
        next_track = min(unvisited, key=lambda t: haversine(race_locations[last], race_locations[t]))
        path.append(next_track)
        unvisited.remove(next_track)
    return path

# -------------------------------
# Step 3: Cluster Centroids
# -------------------------------
cluster_centroids = {}
for cluster_id, tracks in clusters.items():
    lats = [race_locations[t][0] for t in tracks]
    lons = [race_locations[t][1] for t in tracks]
    cluster_centroids[cluster_id] = (np.mean(lats), np.mean(lons))

# -------------------------------
# Step 4: TSP over Clusters
# -------------------------------
def tsp_on_centroids(cluster_order):
    unvisited = cluster_order[:]
    path = [unvisited.pop(0)]
    while unvisited:
        last = path[-1]
        next_cluster = min(unvisited, key=lambda c: haversine(cluster_centroids[last], cluster_centroids[c]))
        path.append(next_cluster)
        unvisited.remove(next_cluster)
    return path

cluster_order = tsp_on_centroids(list(clusters.keys()))

# -------------------------------
# Step 5: Flatten Final Schedule
# -------------------------------
final_schedule = []
for cid in cluster_order:
    intra_cluster_schedule = nearest_neighbor_tsp(clusters[cid])
    final_schedule.extend(intra_cluster_schedule)

# -------------------------------
# Done! Print Final Calendar
# -------------------------------
print("\n🏁 Optimized F1 Calendar:")
for i, race in enumerate(final_schedule, 1):
    print(f"{i:02d}. {race}")



🏁 Optimized F1 Calendar:
01. Sakhir
02. Lusail
03. Yas Island
04. Jeddah
05. Baku
06. Imola
07. Monza
08. Monaco
09. Barcelona
10. Spa-Francorchamps
11. Zandvoort
12. Silverstone
13. Spielberg
14. Budapest
15. Miami
16. Austin
17. Mexico City
18. Las Vegas
19. Montréal
20. São Paulo
21. Melbourne
22. Marina Bay
23. Shanghai
24. Suzuka


In [9]:
len(track_names)

20

In [2]:
%pip install -q numpy scikit-learn
%pip install -q matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import random
import datetime

In [27]:
year = 2030
n = 1

random.seed(42)  # For reproducibility

# Step 1: Generate all Sundays in the year
sundays = []
dt = datetime.date(year, 1, 1)
while dt.year == year:
    if dt.weekday() == 6:  # Sunday
        sundays.append(dt)
    dt += datetime.timedelta(days=1)

In [154]:
def generate_calendar(year: int, n: int) -> list[str]:
    assert 2026 <= year <= 2030, "Year must be between 2026 and 2030"
    assert 15 <= n <= 25, "Races must be between 15 and 25"

    sundays = []
    dt = datetime.date(year, 1, 1)
    while dt.year == year:
        if dt.weekday() == 6:
            sundays.append(dt)
        dt += datetime.timedelta(days=1)

    season_start = max([d for d in sundays if d.month == 2])
    season_end = min([d for d in sundays if d.month == 12])
    sundays = [d for d in sundays if season_start <= d <= season_end]


    sundays = [d for d in sundays if not (d.month == 8 and d.day <= 21)]


    race_days = []
    triple_header_count = 0
    i = 0
    p = 1

    # Determine triple-header constraints
    if n == 25:
        triple_header = 3
    elif n > 21:
        triple_header = 2
    elif n > 19:
        triple_header = 1
    else:
        triple_header = 0

    while len(race_days) < n and i < len(sundays):
        current = sundays[i]

        # Avoid 4-in-a-row
        if len(race_days) >= 3:
            d1, d2, d3 = race_days[-3], race_days[-2], race_days[-1]
            if (
                (d2 - d1).days == 7 and
                (d3 - d2).days == 7 and
                (current - d3).days == 7
            ):
                i += 1
                continue  # skip to avoid 4-in-a-row
        # Insert a triple-header if we're at the beginning or near the end and still allowed
        if (
            triple_header_count < triple_header and
            len(race_days) + 3 <= n and
            i + 2 < len(sundays) and
            p == 0
        ):
            s1, s2, s3 = sundays[i], sundays[i + 1], sundays[i + 2]
            if (s2 - s1).days == 7 and (s3 - s2).days == 7:
                race_days.extend([s1, s2, s3])
                triple_header_count += 1
                i += 3
                p = triple_header_count+1 if triple_header == 3 else triple_header_count+6
                continue

        # Otherwise add this Sunday and the next one with a 2-week gap
        if not race_days or (current - race_days[-1]).days >= 14:
            race_days.append(current)
            if i + 1 < len(sundays):
                race_days.append(sundays[i + 1])
                if triple_header_count < 3:
                    p -= 1  # allow triple-header next time
                else:
                    p = 1
                i += 2
                

        i += 1

    # Truncate to n races
    race_days = race_days[:n]
    return [d.strftime("%d-%m") for d in race_days]

In [165]:
sundays = generate_calendar(2028, 23)

In [166]:
# Find all triplets of consecutive Sundays in the 'sundays' list
consecutive_triplets = [
    sundays[i:i+3]
    for i in range(len(sundays) - 2)
    if (datetime.datetime.strptime(sundays[i+1], "%d-%m").date() - datetime.datetime.strptime(sundays[i], "%d-%m").date()).days == 7 and
       (datetime.datetime.strptime(sundays[i+2], "%d-%m").date() - datetime.datetime.strptime(sundays[i+1], "%d-%m").date()).days == 7
]

# Count the number of triplets
len(consecutive_triplets)
consecutive_triplets

[['19-03', '26-03', '02-04'], ['01-10', '08-10', '15-10']]

In [167]:
len(sundays)

23

In [2]:
a = "[3, 4, 5, 9, 10, 11]"

In [4]:
a = a.strip("[]").split(", ")

In [2]:
# Add the parent directory to the system path to import utils

import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from data.app_data import F1_LOCATION_DESCRIPTIONS
from utils.utilities import get_historical_cities

In [8]:
import os

pwd= os.getcwd()
pwd

'c:\\Users\\massi\\OneDrive\\Desktop\\Uni\\DS-M\\planetf1\\notebooks'

In [3]:


# Fetch historical cities for the year 2026
starting_list = get_historical_cities(year=2025, info=True)

In [4]:
starting_list

,geo_id,code,circuit,city,country,latitude,longitude,first_gp_probability,last_gp_probability
0,5,BAHSAK,Sakhir,Sakhir,Bahrain,26.0325,50.5106,0.230769,0.000000
1,6,AUSMEL,Melbourne,Melbourne,Australia,-37.8497,144.9680,0.730769,0.000000
2,8,CHISHA,Shanghai,Shanghai,China,31.3303,121.2246,0.000000,0.038462
3,7,JAPSUZ,Suzuka,Suzuka,Japan,34.8431,136.5419,0.000000,0.115385
4,24,SAUJED,Jeddah,Jeddah,Saudi Arabia,21.6319,39.1044,0.000000,0.000000
5,30,USAMIA,Miami,Miami,United States,25.9581,-80.2389,0.000000,0.000000
6,16,ITAIMO,Imola,Imola,Italy,44.3439,11.7167,0.000000,0.000000
7,32,MONMON,Monte Carlo,Monte Carlo,Monaco,43.7347,7.4206,0.000000,0.000000
8,10,SPACAT,Catalunya,Montmeló,Spain,41.5700,2.2611,0.000000,0.000000
9,12,CANMON,Montreal,Montreal,Canada,45.5000,-73.5228,0.000000,0.000000


In [54]:
triple_headers=3

In [74]:
random.seed(23)  # For reproducibility

# Ensure there are enough Sundays to pick triple_headers triplets
if len(sundays) < triple_headers * 3:
    raise ValueError("Not enough Sundays to pick the required number of triplets.")

# Find all possible triplets of consecutive Sundays
triplets = [
    sundays[i:i+3]
    for i in range(len(sundays) - 2)
    if (sundays[i+1] - sundays[i]).days == 7 and (sundays[i+2] - sundays[i+1]).days == 7
]

# Randomly pick triple_headers triplets ensuring at least one Sunday from one triplet and another
selected_triplets = []
used_sundays = set()

while len(selected_triplets) < triple_headers:
    triplet = random.choice(triplets)
    if not any(sunday in used_sundays for sunday in triplet):
        selected_triplets.append(triplet)
        used_sundays.update(triplet)
        
        # Add min(date) - 7 days if it exists in sundays
        min_date_minus_7 = min(triplet) - datetime.timedelta(days=7)
        if min_date_minus_7 in sundays:
            used_sundays.add(min_date_minus_7)
        
        # Add max(date) + 7 days if it exists in sundays
        max_date_plus_7 = max(triplet) + datetime.timedelta(days=7)
        if max_date_plus_7 in sundays:
            used_sundays.add(max_date_plus_7)


print("Selected Triplets:")
for triplet in selected_triplets:
    print(triplet)

Selected Triplets:
[datetime.date(2028, 7, 2), datetime.date(2028, 7, 9), datetime.date(2028, 7, 16)]
[datetime.date(2028, 4, 2), datetime.date(2028, 4, 9), datetime.date(2028, 4, 16)]
[datetime.date(2028, 3, 5), datetime.date(2028, 3, 12), datetime.date(2028, 3, 19)]


In [56]:
n=25

In [75]:
final_calendar = []

# Add all Sundays from the selected triplets to the final calendar
for triplet in selected_triplets:
    final_calendar.extend(triplet)

# Ensure the used_sundays set is updated with the triplets
used_sundays.update(final_calendar)

# Randomly pick Sundays from the remaining pool until final_calendar contains n Sundays
remaining_pool = list(set(sundays) - used_sundays)
random.seed(23)  # For reproducibility

while len(final_calendar) < n:
    picked_sunday = random.choice(remaining_pool)
    final_calendar.append(picked_sunday)
    used_sundays.add(picked_sunday)
    remaining_pool.remove(picked_sunday)

    # Remove picked_sunday - 7 days if it exists in the remaining pool
    minus_7 = picked_sunday - datetime.timedelta(days=7)
    minus_14 = picked_sunday - datetime.timedelta(days=14)
    if minus_7 in remaining_pool and minus_14 not in remaining_pool:
        remaining_pool.remove(minus_7)

    # Remove picked_sunday + 7 days if it exists in the remaining pool
    plus_7 = picked_sunday + datetime.timedelta(days=7)
    plus_14 = picked_sunday + datetime.timedelta(days=14)
    if plus_7 in remaining_pool and plus_14 not in remaining_pool:
        remaining_pool.remove(plus_7)
        
    print(f"Picked Sunday: {picked_sunday}")
    print(f"Remaining Pool: {remaining_pool}")
    print(f"Final Calendar length: {len(final_calendar)}")

# Sort the final calendar for chronological order
final_calendar.sort()

print("Final Calendar:")
print(f"Total Sundays: {len(final_calendar)}")
for sunday in final_calendar:
    print(sunday)

Picked Sunday: 2028-05-07
Remaining Pool: [datetime.date(2028, 5, 21), datetime.date(2028, 11, 26), datetime.date(2028, 6, 18), datetime.date(2028, 6, 4), datetime.date(2028, 10, 1), datetime.date(2028, 9, 24), datetime.date(2028, 10, 8), datetime.date(2028, 10, 15), datetime.date(2028, 10, 29), datetime.date(2028, 5, 14), datetime.date(2028, 5, 28), datetime.date(2028, 6, 11), datetime.date(2028, 7, 30), datetime.date(2028, 9, 17), datetime.date(2028, 9, 3), datetime.date(2028, 11, 12), datetime.date(2028, 11, 5), datetime.date(2028, 10, 22), datetime.date(2028, 11, 19), datetime.date(2028, 8, 27), datetime.date(2028, 9, 10)]
Final Calendar length: 10
Picked Sunday: 2028-06-18
Remaining Pool: [datetime.date(2028, 5, 21), datetime.date(2028, 11, 26), datetime.date(2028, 6, 4), datetime.date(2028, 10, 1), datetime.date(2028, 9, 24), datetime.date(2028, 10, 8), datetime.date(2028, 10, 15), datetime.date(2028, 10, 29), datetime.date(2028, 5, 14), datetime.date(2028, 5, 28), datetime.date(

IndexError: Cannot choose from an empty sequence

Final Race Days:
2028-03-05
2028-03-12
2028-03-19
2028-04-02
2028-04-09
2028-04-16
2028-05-14
2028-05-21
2028-06-04
2028-06-18
2028-07-02
2028-07-09
2028-07-16
2028-07-30
2028-08-27
2028-09-10
2028-09-17
2028-09-24
2028-10-08
2028-10-15
2028-10-22
2028-10-29
2028-11-12
2028-11-19
2028-11-26


![alt text](image.png)

In [30]:
if n == 25:
    min_triple_header = 3
    max_triple_header = 3
elif n > 21:
    min_triple_header = 2
    max_triple_header = 2
elif n > 19:
    min_triple_header = 1
    max_triple_header = 2
else:
    min_triple_header = 0
    max_triple_header = 2
    

In [31]:
race_days = []
triple_header_count = 0

2025-05-03 21:41:52.823 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-03 21:41:53.325 
  command:

    streamlit run C:\Users\massi\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-03 21:41:53.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-03 21:41:53.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-03 21:41:53.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-03 21:41:53.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-03 21:41:53.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-03 21:41:53.325 Thre

In [6]:
import os
import re
from collections import defaultdict

def extract_imports_from_file(file_path):
    """Extracts all imported modules from a Python file."""
    imports = set()
    with open(file_path, 'r') as file:
        for line in file:
            # Match 'import module' or 'from module import ...'
            match = re.match(r'^\s*(?:import|from)\s+([\w\.]+)', line)
            if match:
                imports.add(match.group(1).split('.')[0])  # Get the top-level module
    return imports

def find_requirements_in_project(start_path='.'):
    """Finds all imported modules in Python files within the project."""
    requirements = defaultdict(set)
    for root, _, files in os.walk(start_path):
        for file in files:
            if file.endswith('.py'):  # Only process Python files
                file_path = os.path.join(root, file)
                imports = extract_imports_from_file(file_path)
                requirements[file_path].update(imports)
    return requirements

# Find all requirements in the project
project_requirements = find_requirements_in_project()

# Flatten the requirements into a single list
all_requirements = set()
for file, imports in project_requirements.items():
    all_requirements.update(imports)

# Print the requirements
print("Requirements:")
for requirement in sorted(all_requirements):
    print(requirement)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 1803: character maps to <undefined>

In [4]:
import os

def print_folder_tree_md(start_path='.'):
    result = []
    result.append("# Folder Tree\n")
    for root, dirs, files in os.walk(start_path):
        # Filter out hidden directories and __pycache__ folders
        dirs[:] = [d for d in dirs if not d.startswith('.') and d != '__pycache__']
        level = root.replace(start_path, '').count(os.sep)
        indent = '  ' * level
        result.append(f"{indent}- **{os.path.basename(root)}/**")
        sub_indent = '  ' * (level + 1)
        for file in files:
            # Include only non-hidden files or .gitignore
            if not file.startswith('.') or file == '.gitignore':
                result.append(f"{sub_indent}- {file}")
    return '\n'.join(result)

# Generate the folder tree in README.md format
folder_tree_md = print_folder_tree_md()
print(folder_tree_md)

# Folder Tree

- **./**
  - .gitignore
  - clustering.ipynb
  - clustering_results.docx
  - Comprehensive Data Requirements for.docx
  - data_collection.ipynb
  - data_prep.ipynb
  - genetic.ipynb
  - genetic_exe.ipynb
  - image.png
  - math.ipynb
  - planet_fone.db
  - readme.md
  - requirements.txt
  - training_regression_calendar.csv
  - **data/**
    - calendar.csv
    - Costraints.xlsx
    - fone_calendar.csv
    - fone_geography.csv
    - sqlite_sequence.csv
    - travel_logistic.csv
  - **fastf1_cache/**
    - fastf1_http_cache.sqlite
  - **models/**
    - clustering.py
    - genetic_ops.py
    - __init__.py
  - **utils/**
    - sql.py
    - utilities.py
    - __init__.py


# F1 Green Flag - A Sustainable Data Science Solution for F1 Calendar Management

## Description

This project analyzes Formula 1 logistics data to develop actionable recommendations for reducing the sport's environmental impact. By leveraging data science techniques, including clustering, regression analysis, and genetic algorithms, we aim to optimize the F1 race calendar, specifically targeting the 2026 season, to minimize travel-related emissions.

The core of the optimization uses a Genetic Algorithm (built with the DEAP library) to find near-optimal sequences of race locations, minimizing a fitness function based on travel distance (as a proxy for emissions) between consecutive events, while potentially considering other logistical constraints.

## Project Goal

To provide data-driven insights and a computationally optimized race calendar proposal for the 2026 Formula 1 season, demonstrably reducing the carbon footprint associated with logistics and travel compared to traditional scheduling approaches.

## Features

* **Data Collection & Preparation:** Scripts and notebooks to gather F1 circuit, geographical, and logistical data, culminating in the `planet_fone.db` database.
* **Data Analysis:** Jupyter notebooks exploring the data using techniques like clustering (`clustering.ipynb`) and potentially regression (`training_regression_calendar.csv` suggests this).
* **Genetic Algorithm Optimization:** Implementation of a custom Genetic Algorithm (`genetic.ipynb`, `genetic_ops.py`) using DEAP to find optimized race sequences based on minimizing travel distance/emissions.
* **Modular Structure:** Code organized into utilities (`utils/`), models/operators (`models/`), and analysis notebooks.

## Installation

1.  **Prerequisites:**
    * Python 3.12.5 or later.
    * `pip` (Python package installer).

2.  **Clone the repository (if applicable):**
    ```bash
    git clone <your-repository-url>
    cd <your-project-directory>
    ```

3.  **Install Dependencies:**
    Install the required Python packages using the provided `requirements.txt` file:
    ```bash
    pip install -r requirements.txt
    ```

## Usage

Currently, the project primarily consists of Jupyter notebooks for data processing, analysis, and model development/tuning.

* **Data Collection:** Run `data_collection.ipynb` to gather initial data.
* **Data Preparation:** Run `data_prep.ipynb` to clean and prepare data for analysis and modeling.
* **Clustering Analysis:** Explore `clustering.ipynb`.
* **Genetic Algorithm:**
    * `genetic.ipynb`: Contains development, testing, and tuning of the GA components.
    * `genetic_exe.ipynb`: Executes the tuned Genetic Algorithm to find optimized calendars.

*(Note: A main executable script for running the entire pipeline or specific tasks is under development.)*

## File Structure


```text
./
├── .gitignore                  # Ignored files for git
├── clustering.ipynb            # Clustering module testing and tuning
├── data_collection.ipynb       # For gathering data and creation of planet_fone.db
├── data_prep.ipynb             # Data prep for regression, clustering, and genetic algo
├── genetic.ipynb               # Genetic module building and tuning
├── genetic_exe.ipynb           # Genetic module execution
├── image.png                   # Used in data prep notebook
├── math.ipynb                  # Testing notebook
├── planet_fone.db              # Main datasource SQLite database
├── readme.md                   # This README file
├── requirements.txt            # Python package dependencies
├── training_regression_calendar.csv # Initial dataset possibly for regression
│
├── data/                       # Data files, backups, constraints
│   ├── calendar.csv
│   ├── Costraints.xlsx         # Constraints for scheduling?
│   ├── fone_calendar.csv
│   ├── fone_geography.csv
│   ├── sqlite_sequence.csv
│   └── travel_logistic.csv
│
├── fastf1_cache/               # Cache for the FastF1 library
│   └── fastf1_http_cache.sqlite
│
├── models/                     # Custom model implementations (operators, etc.)
│   ├── clustering.py           # Clustering operators/functions
│   ├── genetic_ops.py          # Genetic Algorithm operators (crossover, mutation, fitness)
│   └── __init__.py
│
└── utils/                      # Utility scripts
    ├── sql.py                  # SQL database utilities
    ├── utilities.py            # Generic utility functions
    └── __init__.py

## Data

The primary data source is the SQLite database `planet_fone.db`, which is created and populated by the `data_collection.ipynb` and `data_prep.ipynb` notebooks. Additional raw or intermediate data files, as well as potential constraint definitions, are located in the `data/` directory.

## Contributing

This project is developed by:

* Jakob Jonas Spranger ([jakobjspranger](https://github.com/jakobjspranger))
* Juan Jose Montesinos ([juanjomontesinos](https://github.com/juanjomontesinos))
* Maximilian von Braun ([MaxBraun-EAE](https://github.com/MaxBraun-EAE))
* Massimiliano Napolitano ([Ulquiorra-23](https://github.com/Ulquiorra-23))

*(Feel free to add contribution guidelines if desired)*

## License

*(Specify your project's license here, e.g., MIT License, Apache 2.0, or leave blank if undecided.)*

